In [1]:
import random
import gym
import numpy as np
from collections import deque
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Conv2D, MaxPooling2D
from keras.optimizers import Adam

/Users/tim/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# Agent

In [2]:
class DQN_Agent:
    #
    # Initializes attributes and constructs CNN model and target_model
    #
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=5000)
        
        # Hyperparameters
        self.gamma = 0.9            # Discount rate
        self.epsilon = 1.0          # Exploration rate
        self.epsilon_min = 0.1      # Minimal exploration rate (epsilon-greedy)
        self.epsilon_decay = 0.995  # Decay rate for epsilon
        self.update_rate = 1000     # Number of steps until updating the target network
        
        # Construct DQN models
        self.model = self._build_model()
        self.target_model = self._build_model()
        self.target_model.set_weights(self.model.get_weights())
        self.model.summary()

    #
    # Constructs CNN
    #
    def _build_model(self):
        model = Sequential()
        
        # Conv Layers
        model.add(Conv2D(32, (8, 8), strides=4, padding='same', input_shape=self.state_size))
        model.add(Activation('relu'))
        
        model.add(Conv2D(64, (4, 4), strides=2, padding='same'))
        model.add(Activation('relu'))
        
        model.add(Conv2D(64, (3, 3), strides=1, padding='same'))
        model.add(Activation('relu'))
        model.add(Flatten())

        # FC Layers
        model.add(Dense(512, activation='relu'))
        model.add(Dense(self.action_size, activation='linear'))
        
        model.compile(loss='mse', optimizer=Adam())
        return model

    #
    # Stores experience in replay memory
    #
    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    #
    # Chooses action based on epsilon-greedy policy
    #
    def act(self, state):
        # Random exploration
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        
        act_values = self.model.predict(state)
        
        return np.argmax(act_values[0])  # Returns action using policy

    #
    # Trains the model using randomly selected experiences in the replay memory
    #
    def replay(self, batch_size):
        minibatch = random.sample(self.memory, batch_size)
        
        for state, action, reward, next_state, done in minibatch:
            
            if not done:
                target = (reward + self.gamma * np.amax(self.target_model.predict(next_state)))
            else:
                target = reward
                
            # Construct the target vector as follows:
            # 1. Use the current model to output the Q-value predictions
            target_f = self.model.predict(state)
            
            # 2. Rewrite the chosen action value with the computed target
            target_f[0][action] = target
            
            # 3. Use vectors in the objective computation
            self.model.fit(state, target_f, epochs=1, verbose=0)
            
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

    #
    # Sets the target model parameters to the current model parameters
    #
    def update_target_model(self):
        self.target_model.set_weights(self.model.get_weights())
            
    #
    # Loads a saved model
    #
    def load(self, name):
        self.model.load_weights(name)

    #
    # Saves parameters of a trained model
    #
    def save(self, name):
        self.model.save_weights(name)

# Preprocessing

In [3]:
# Helpful preprocessing taken from github.com/ageron/tiny-dqn
def process_frame(frame):
    mspacman_color = np.array([210, 164, 74]).mean()
    img = frame[1:176:2, ::2]    # Crop and downsize
    img = img.mean(axis=2)       # Convert to greyscale
    img[img==mspacman_color] = 0 # Improve contrast by making pacman white
    img = (img - 128) / 128 - 1  # Normalize from -1 to 1.
    
    return np.expand_dims(img.reshape(88, 80, 1), axis=0)

In [4]:
def blend_images(images, blend):
    avg_image = np.expand_dims(np.zeros((88, 80, 1), np.float64), axis=0)

    for image in images:
        avg_image += image
        
    if len(images) < blend:
        return avg_image / len(images)
    else:
        return avg_image / blend

# Environment

In [5]:
env = gym.make('MsPacman-v0')
state_size = (88, 80, 1)
action_size = env.action_space.n
agent = DQN_Agent(state_size, action_size)

episodes = 500
batch_size = 8
skip_start = 90  # MsPacman-v0 waits for 90 actions before the episode begins
total_time = 0   # Counter for total number of steps taken
all_rewards = 0  # Used to compute avg reward over time
blend = 4        # Number of images to blend
done = False

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 22, 20, 32)        2080      
_________________________________________________________________
activation_1 (Activation)    (None, 22, 20, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 11, 10, 64)        32832     
_________________________________________________________________
activation_2 (Activation)    (None, 11, 10, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 11, 10, 64)        36928     
_________________________________________________________________
activation_3 (Activation)    (None, 11, 10, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 7040)              0         
__________

In [ ]:
for e in range(episodes):
    total_reward = 0
    game_score = 0
    state = process_frame(env.reset())
    images = deque(maxlen=blend)  # Array of images to be blended
    images.append(state)
    
    for skip in range(skip_start): # skip the start of each game
        env.step(0)
    
    for time in range(20000):
        env.render()
        total_time += 1
        
        # Every update_rate timesteps we update the target network parameters
        if total_time % agent.update_rate == 0:
            agent.update_target_model()
        
        # Return the avg of the last 4 frames
        state = blend_images(images, blend)
        
        # Transition Dynamics
        action = agent.act(state)
        next_state, reward, done, _ = env.step(action)
        
        # Return the avg of the last 4 frames
        next_state = process_frame(next_state)
        images.append(next_state)
        next_state = blend_images(images, blend)
        
        # Store sequence in replay memory
        agent.remember(state, action, reward, next_state, done)
        
        state = next_state
        game_score += reward
        reward -= 1  # Punish behavior which does not accumulate reward
        total_reward += reward
        
        if done:
            all_rewards += game_score
            
            print("episode: {}/{}, game score: {}, reward: {}, avg reward: {}, time: {}, total time: {}"
                  .format(e+1, episodes, game_score, total_reward, all_rewards/(e+1), time, total_time))
            
            break
            
        if len(agent.memory) > batch_size:
            agent.replay(batch_size)

episode: 1/500, game score: 620.0, reward: -208.0, avg reward: 620.0, time: 827, total time: 828
episode: 2/500, game score: 560.0, reward: -370.0, avg reward: 590.0, time: 929, total time: 1758
episode: 3/500, game score: 870.0, reward: -160.0, avg reward: 683.333333333, time: 1029, total time: 2788
episode: 4/500, game score: 420.0, reward: -168.0, avg reward: 617.5, time: 587, total time: 3376
episode: 5/500, game score: 540.0, reward: -177.0, avg reward: 602.0, time: 716, total time: 4093
episode: 6/500, game score: 380.0, reward: -122.0, avg reward: 565.0, time: 501, total time: 4595
episode: 7/500, game score: 330.0, reward: -102.0, avg reward: 531.428571429, time: 431, total time: 5027
episode: 8/500, game score: 420.0, reward: -10.0, avg reward: 517.5, time: 429, total time: 5457
episode: 9/500, game score: 190.0, reward: -111.0, avg reward: 481.111111111, time: 300, total time: 5758
episode: 10/500, game score: 580.0, reward: -115.0, avg reward: 491.0, time: 694, total time: 6

episode: 79/500, game score: 330.0, reward: -59.0, avg reward: 597.848101266, time: 388, total time: 53784
episode: 80/500, game score: 1110.0, reward: -296.0, avg reward: 604.25, time: 1405, total time: 55190
episode: 81/500, game score: 620.0, reward: -67.0, avg reward: 604.444444444, time: 686, total time: 55877
episode: 82/500, game score: 480.0, reward: -132.0, avg reward: 602.926829268, time: 611, total time: 56489
episode: 83/500, game score: 300.0, reward: -192.0, avg reward: 599.277108434, time: 491, total time: 56981
episode: 84/500, game score: 300.0, reward: -179.0, avg reward: 595.714285714, time: 478, total time: 57460
episode: 85/500, game score: 1610.0, reward: 773.0, avg reward: 607.647058824, time: 836, total time: 58297
episode: 86/500, game score: 180.0, reward: -226.0, avg reward: 602.674418605, time: 405, total time: 58703
episode: 87/500, game score: 370.0, reward: -451.0, avg reward: 600.0, time: 820, total time: 59524
episode: 88/500, game score: 320.0, reward:

episode: 155/500, game score: 650.0, reward: -151.0, avg reward: 605.741935484, time: 800, total time: 104030
episode: 156/500, game score: 540.0, reward: -150.0, avg reward: 605.320512821, time: 689, total time: 104720
episode: 157/500, game score: 350.0, reward: -238.0, avg reward: 603.694267516, time: 587, total time: 105308
episode: 158/500, game score: 970.0, reward: -261.0, avg reward: 606.012658228, time: 1230, total time: 106539
episode: 159/500, game score: 450.0, reward: -86.0, avg reward: 605.031446541, time: 535, total time: 107075
episode: 160/500, game score: 620.0, reward: -21.0, avg reward: 605.125, time: 640, total time: 107716
episode: 161/500, game score: 390.0, reward: -355.0, avg reward: 603.788819876, time: 744, total time: 108461
episode: 162/500, game score: 410.0, reward: -43.0, avg reward: 602.592592593, time: 452, total time: 108914
episode: 163/500, game score: 1020.0, reward: 21.0, avg reward: 605.153374233, time: 998, total time: 109913
episode: 164/500, g

In [11]:
agent.save('models/5k-memory_500-games')